In [1]:
import pandas as pd
import numpy as np
import re
import string
import joblib
import datetime as dt
import html
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [2]:
# Load data
dfRaw=pd.read_csv('sampleFinal.csv')
print(dfRaw)

       Unnamed: 0    User_id  \
0               0  351225728   
1               1  351224292   
2               2  351217203   
3               3  351168082   
4               4  351101967   
...           ...        ...   
58867       58867    6679527   
58868       58868    6678652   
58869       58869    6675507   
58870       58870    6667334   
58871       58871    6667230   

                                                 Message       Date  \
0      $IDEX  the groups all showing up as long as th...   1/7/2021   
1      $SPY $DIA $QQQ $DJIA $NASDAQ  not only Bofa an...   1/7/2021   
2      $EURUSD the major key avent is -Fed to announc...   1/7/2021   
3      $CCL $RCL $SPY $QQQ $NASDAQ Im moving to the b...   1/7/2021   
4      It’s strange.. once you get used to making 1k ...  30/6/2021   
...                                                  ...        ...   
58867  RT @vcutrader The Nasdaq is up 11.4% in 34 day...   3/2/2012   
58868  Short $DJIA 12.850 $NDX 2,527 $RUT 829 $

In [3]:
# Inspect the number of observations
dfRaw.shape

(58872, 6)

In [4]:
dfRaw.head

<bound method NDFrame.head of        Unnamed: 0    User_id  \
0               0  351225728   
1               1  351224292   
2               2  351217203   
3               3  351168082   
4               4  351101967   
...           ...        ...   
58867       58867    6679527   
58868       58868    6678652   
58869       58869    6675507   
58870       58870    6667334   
58871       58871    6667230   

                                                 Message       Date  \
0      $IDEX  the groups all showing up as long as th...   1/7/2021   
1      $SPY $DIA $QQQ $DJIA $NASDAQ  not only Bofa an...   1/7/2021   
2      $EURUSD the major key avent is -Fed to announc...   1/7/2021   
3      $CCL $RCL $SPY $QQQ $NASDAQ Im moving to the b...   1/7/2021   
4      It’s strange.. once you get used to making 1k ...  30/6/2021   
...                                                  ...        ...   
58867  RT @vcutrader The Nasdaq is up 11.4% in 34 day...   3/2/2012   
58868  Short $DJI

In [5]:
list(dfRaw.columns)

['Unnamed: 0', 'User_id', 'Message', 'Date', 'Time', 'Sentiment']

In [6]:
# Keep the required columns for processing
dfRawDrop=dfRaw[['Message','Sentiment']]
print(dfRawDrop)

                                                 Message Sentiment
0      $IDEX  the groups all showing up as long as th...   Bullish
1      $SPY $DIA $QQQ $DJIA $NASDAQ  not only Bofa an...   Bearish
2      $EURUSD the major key avent is -Fed to announc...   Bearish
3      $CCL $RCL $SPY $QQQ $NASDAQ Im moving to the b...       NaN
4      It’s strange.. once you get used to making 1k ...       NaN
...                                                  ...       ...
58867  RT @vcutrader The Nasdaq is up 11.4% in 34 day...       NaN
58868  Short $DJIA 12.850 $NDX 2,527 $RUT 829 $SPX 1....       NaN
58869  $IJR  $COMPQ  Both indexes have confirmed the ...       NaN
58870                                           YEA $NDX       NaN
58871  We&#39;ve Added The NASDAQ 100 Index Stream (T...       NaN

[58872 rows x 2 columns]


In [7]:
# Check any imbalance in dataset

dfRawDrop.groupby('Sentiment').size()

Sentiment
Bearish    6763
Bullish    9708
dtype: int64

In [22]:
# Drop unlabelled rows

dfRawDropNA=dfRawDrop.dropna()
print(dfRawDropNA)

                                                 Message Sentiment
0      $IDEX  the groups all showing up as long as th...   Bullish
1      $SPY $DIA $QQQ $DJIA $NASDAQ  not only Bofa an...   Bearish
2      $EURUSD the major key avent is -Fed to announc...   Bearish
6      July has been a kind month for the NASDAQ in t...   Bullish
7      $SPY $DIA $QQQ $RUT china based companies gett...   Bullish
...                                                  ...       ...
58731                           $NDX http://stks.co/2iHC   Bearish
58783                   $NDX http://stks.co/2Ypo -Weekly   Bullish
58810                   $NDX http://stks.co/2TWl -Weekly   Bullish
58859  $NDX http://stks.co/2Jh3 Nasdaq-100 Bullish Pe...   Bullish
58866            $NDX http://stks.co/2FXs -Weekly-Strong   Bullish

[16471 rows x 2 columns]


In [23]:
# Clean data (lower case, remove special characters and numbers, weblinks)

dfRawDropNA['MessageCleaned'] = dfRawDropNA['Message'].str.lower()
dfRawDropNA['MessageCleaned'] = dfRawDropNA['MessageCleaned'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
print(dfRawDropNA)

<ipython-input-23-660219ac20b5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRawDropNA['MessageCleaned'] = dfRawDropNA['Message'].str.lower()
                                                 Message Sentiment  \
0      $IDEX  the groups all showing up as long as th...   Bullish   
1      $SPY $DIA $QQQ $DJIA $NASDAQ  not only Bofa an...   Bearish   
2      $EURUSD the major key avent is -Fed to announc...   Bearish   
6      July has been a kind month for the NASDAQ in t...   Bullish   
7      $SPY $DIA $QQQ $RUT china based companies gett...   Bullish   
...                                                  ...       ...   
58731                           $NDX http://stks.co/2iHC   Bearish   
58783                   $NDX http://stks.co/2Ypo

In [24]:
dfRawDropNA.to_csv('sampleFinalCleaned.csv')

In [25]:
# Remove common stopwords
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
dfRawDropNA['MessageCleaned'] = dfRawDropNA['MessageCleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
dfRawDropNA.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BSIB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-25-62adf75aa5e5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRawDropNA['MessageCleaned'] = dfRawDropNA['MessageCleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


,Message,Sentiment,MessageCleaned
0,$IDEX the groups all showing up as long as th...,Bullish,idex groups showing long djia nasdaq spy qqq l...
1,$SPY $DIA $QQQ $DJIA $NASDAQ not only Bofa an...,Bearish,spy dia qqq djia nasdaq bofa wells fargo predi...
2,$EURUSD the major key avent is -Fed to announc...,Bearish,eurusd major key avent fed announce qe taper a...
6,July has been a kind month for the NASDAQ in t...,Bullish,july kind month nasdaq past 5 years compq nasd...
7,$SPY $DIA $QQQ $RUT china based companies gett...,Bullish,spy dia qqq rut china based companies getting ...


In [26]:
dfRawDropNA.to_csv('sampleFinalCleanedStopWords.csv')

In [27]:
words = nltk.tokenize.word_tokenize(dfRawDropNA['MessageCleaned'].str.cat(sep=' '))
word_dist = nltk.FreqDist(words)
print (word_dist)

<FreqDist with 22393 samples and 236612 outcomes>


In [29]:
# Check top common words that can be removed
freqDist = pd.DataFrame(word_dist.most_common(10),columns=['Word', 'Frequency'])
print(freqDist)

      Word  Frequency
0   nasdaq      11039
1      spy       9502
2      ndx       7346
3     djia       6087
4      qqq       4471
5      spx       3870
6   market       1719
7      dia       1496
8     aapl       1217
9  bullish       1118


In [30]:
addStop = ['nasdaq','spy','ndx','djia','qqq']
dfRawDropNA['MessageCleaned'] = dfRawDropNA['MessageCleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (addStop)]))
dfRawDropNA.head()

<ipython-input-30-debb33e44fbb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRawDropNA['MessageCleaned'] = dfRawDropNA['MessageCleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (addStop)]))


,Message,Sentiment,MessageCleaned
0,$IDEX the groups all showing up as long as th...,Bullish,idex groups showing long looks good undervalue...
1,$SPY $DIA $QQQ $DJIA $NASDAQ not only Bofa an...,Bearish,dia bofa wells fargo predicts us 10 years yiel...
2,$EURUSD the major key avent is -Fed to announc...,Bearish,eurusd major key avent fed announce qe taper a...
6,July has been a kind month for the NASDAQ in t...,Bullish,july kind month past 5 years compq
7,$SPY $DIA $QQQ $RUT china based companies gett...,Bullish,dia rut china based companies getting impressi...


In [31]:
dfRawDropNA.to_csv('sampleFinalCleanedStopWords.csv')

In [32]:
dfRawDropNA.groupby('MessageCleaned').size()

MessageCleaned
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    228
0 soon everyone flock dogex part plan                                                                                                                                                                                                                                                                                                                                                                                                                                                 1
01000010 01100101 01100101 01110000 00100

In [34]:
dfRawDropNA.shape

(16471, 3)

In [36]:
# Convert blank fields to NaN
nan_value = float("NaN")

dfRawDropNA.replace("", nan_value, inplace=True)

C:\Users\BSIB\anaconda3\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [38]:
dfRawDropNA.shape

(16471, 3)

In [39]:
# Remove NaN
dfCleaned=dfRawDropNA.dropna()
dfCleaned.shape

(16243, 3)

,Message,Sentiment,MessageCleaned
0,$IDEX the groups all showing up as long as th...,Bullish,idex groups showing long looks good undervalue...
1,$SPY $DIA $QQQ $DJIA $NASDAQ not only Bofa an...,Bearish,dia bofa wells fargo predicts us 10 years yiel...
2,$EURUSD the major key avent is -Fed to announc...,Bearish,eurusd major key avent fed announce qe taper a...
6,July has been a kind month for the NASDAQ in t...,Bullish,july kind month past 5 years compq
7,$SPY $DIA $QQQ $RUT china based companies gett...,Bullish,dia rut china based companies getting impressi...
...,...,...,...
58729,$NDX http://stks.co/2itL #stockmarket looks po...,Bearish,stockmarket looks poised resume sell couple da...
58783,$NDX http://stks.co/2Ypo -Weekly,Bullish,weekly
58810,$NDX http://stks.co/2TWl -Weekly,Bullish,weekly
58859,$NDX http://stks.co/2Jh3 Nasdaq-100 Bullish Pe...,Bullish,nasdaq100 bullish percent index ended today 82...


In [41]:
dfCleaned.to_csv('sampleToBeVect.csv')